# User interface

In [1]:
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import preprocess_functions
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
reload(preprocess_functions)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd


## Functions

### Misc functions

In [2]:
def display_markdown():
    # This function displays the markdown text, project_dict is 
    # a dictionary containing the project information
    markdown_text = f"""
    ### Experiment Data
    **User:** {project_dict['User']}  \n
    **Dataset:** {project_dict['Dataset']}  \n
    **Session:** {project_dict['Session']}  \n
    **Task:** {project_dict['Task']}  \n
    **Participants:** {project_dict['Participants']}  \n
    **Runs:** {project_dict['Runs']}  \n
    **Specie:** {project_dict['Specie']}  
    """

def create_input_widgets():
    input_widgets = {
        "User": widgets.Text(value='raulh87', description='User:'),
        "Dataset": widgets.Text(value='Complex', description='Dataset:'),
        "Session": widgets.Text(value='', description='Session (optional):'),
        "Task": widgets.Text(value='visual', description='Task:'),
        "Participants": widgets.Text(value='1,2,3,4,5', description='Participant numbers:'),
        "Runs": widgets.Text(value='1,2,3', description='Runs available (1,2,3,...)'),
        "Specie": widgets.Text(value='D', description='Specie (D or H):'),
    }
    return input_widgets

def update_data(b):
    global project_dict
    #clear_output(wait=True)
    for key, widget in input_widgets.items():
        if key == 'Participants' or key == 'Runs':
            project_dict[key] = [int(i) for i in widget.value.split(',')]
        else:
            project_dict[key] = widget.value

    # update the project_dict in the screen
    update_project_dict()

def update_project_dict():
    row_2.clear_output()
    for key, value in project_dict.items():
        #print(f"{key}: {value}")
        with row_2:
            print(f"{key}: {value}")


def cancel_update(b):
    #clear_output(wait=True)
    for key, widget in input_widgets.items():
        widget.value = project_dict[key]
    
    # update the project_dict in the screen
    update_project_dict()


# process_list as dictionary process_dict
process_dict = {
    'preprocess_run': {'name': 'Preprocess', 'by_run': True, 'process_used': 'preprocess_run'},
    'get_mean_fct': {'name': 'Mean fct', 'by_run': False, 'process_used': 'get_mean_fct'},
    #'crop_interface': {'name': 'Crop', 'by_run': False, 'process_used': 'crop_interface'},
    #'bet_interface': {'name': 'BET', 'by_run': False, 'process_used': 'bet_interface'},
    'mean_to_STD': {'name': 'Mean to atlas', 'by_run': False, 'process_used': 'mean_to_STD'},
    'run_to_STD': {'name': 'Runs to atlas', 'by_run': True, 'process_used': 'run_to_STD'},
    'process_motion': {'name': 'Motion', 'by_run': False, 'process_used': 'process_motion'},
}

process_name_list = [process[1]['name'] for process in process_dict.items()]

### Running the app

In [3]:
# Define the dictionary and the display functions
reload(utils)
reload(preprocess_functions)

project_dict = {
    "User": "",
    "Dataset": "",
    "Session": "",
    "Task": "",
    "Participants": "",
    "Runs": "",
    "Specie": "",
}
project_dict = {
    "User": "raulh87",
    "Dataset": "Complex",
    "Session": "",
    "Task": "visual",
    "Participants": [1,2,3,4,5],
    "Runs": [1,2,3],
    "Specie": "D",
}
if project_dict['User']:
    username = project_dict['User']
    if os.name == 'nt':
        datafolder = r"P:\userdata" + os.sep + username + r"\data"
    else:
        datafolder = '/home' + os.sep + username + '/mnt/a471/userdata/' + username + '/data'
else:
    datafolder = ''
# assign to project dict
project_dict['Datafolder'] = datafolder
# Create a list to store the jobs
job_list = [] 

# Create the input widgets
input_widgets = create_input_widgets()
update_button = widgets.Button(description="Update")
cancel_button = widgets.Button(description="Cancel")

# Assign an event to the button
update_button.on_click(update_data)
cancel_button.on_click(cancel_update)

# Create a tab widget
tab = widgets.Tab()

# Create the rows to store the widgets
row_1 = widgets.VBox([widgets.VBox(list(input_widgets.values())), update_button, cancel_button])
row_2 = widgets.Output(layout={'border': '1px solid black'})                     

# Create the columns to store in tab 1
data_entry_tab = widgets.HBox([row_1,row_2])

#participant_list = [f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']]

# Create the sub_selector dropdown for participants
sub_selector = widgets.Dropdown(
    options=[f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']],
    value=f"{project_dict['Specie']}-sub-{project_dict['Participants'][0]:03d}",
    description='Participants:',
    disabled=False,
)

# Create the process_selector ToggleButtons for process based on process_list
process_selector = widgets.ToggleButtons(
    options=[process[1]['name'] for process in process_dict.items()],
    description='Process:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # icons=['check'] * 3
)

# Create checkbox to select the runs to process
run_selector = widgets.HBox(
    [widgets.Checkbox(value=True, description=f"Run {i}") for i in project_dict['Runs']]
)


# Button to schedule the process, make the button black with white text
run_button = widgets.Button(
    description='Add job',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add selected process to the job list   ',
    icon='book',
)

run_jobs_button = widgets.Button(
    description='Run jobs',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run the selected jobs',
    icon='mail-forward'
)

# Create job_dict to store the process information
job_dict = {
    "User": project_dict['User'],
    "Dataset": project_dict['Dataset'],
    "Session": project_dict['Session'],
    "Task": project_dict['Task'],
    "sub_N":  int(sub_selector.value.split('-')[-1]),
    "run_N": [i+1 for i, run in enumerate(run_selector.children) if run.value],
    "Specie": project_dict['Specie'],
    "Process": process_selector.value,
    "Datafolder": project_dict['Datafolder'],
    "Status": "Scheduled",
    "Atlas_type": 'Czeibert',
    "Combination": ['-x','z','-y'],
    "Smooth": 0,
}

# create empty schedule_table
schedule_table = pd.DataFrame()
html_table = schedule_table.to_html(classes='table table-striped')
table_widget = widgets.HTML(value=html_table)

def update_sub(b):
    global job_dict
    job_dict['sub_N'] = int(sub_selector.value.split('-')[-1])

def process_selector_button_pressed(b):
    global job_dict
    # get the key of the process selected matching process_selector.value with process_name_list
    # process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]
    process_key = [key for key, value in process_dict.items() if value['name'] == process_selector.value][0]

    job_dict['Process'] = process_selector.value
    #print(job_dict['Process'])
    # check if the process is by run
    if process_dict[process_key]['by_run']:
        run_selector.children = [widgets.Checkbox(value=True, description=f"Run {i}") for i in project_dict['Runs']]
    else:
        run_selector.children = [widgets.Checkbox(value=True, description=f"Run {i}", disabled=True) for i in project_dict['Runs']]
    # update job_dict
    job_dict['run_N'] = [i+1 for i, run in enumerate(run_selector.children) if run.value]
    #print(job_dict)

def update_runs(b):
    global job_dict
    job_dict['run_N'] = [i+1 for i, run in enumerate(run_selector.children) if run.value]
    #print(job_dict['run_N'])

# Create function for run_button
def add_job_button_pressed(b):
    global job_dict, job_list, schedule_table
    # clear the output
    #clear_output(wait=True)
    #Output.clear_output()
        # print the job_dict

    # add the job_dict to the job_list
    job_list.append(job_dict.copy())
    #print('job added number of jobs: ' + str(len(job_list)))
    # update the schedule_table
    schedule_table = utils.job_list_to_table(job_list)
    html_table = schedule_table.to_html(classes='table table-striped')
    # update the table_widget
    table_widget.value = html_table

def crop_app_pushed():
    tab_crop_app,crop_app_out = preprocess_functions.crop_app(project_dict, job_dict['sub_N'])
    with crop_app_out:
        # Clear previous output
        crop_app_out.clear_output()
        #print('supposed to be cleaned')
    display(crop_app_out)

def bet_app_pushed():
    bet_app_tab, bet_app_out = preprocess_functions.bet_app(project_dict, job_dict['sub_N'])
    with bet_app_out:
        # Clear previous output
        bet_app_out.clear_output()
    display(bet_app_out)

def other_pushed():
    with crop_app_out:
        # Clear previous output
        crop_app_out.clear_output()
    

def run_jobs(b):
    global job_list, schedule_table
    for n,job in enumerate(job_list):
        #print(job)
        # check status of the job
        if job_list[n]['Status'] == 'Finished':
            continue
        else:
            preprocess_functions.run_process(job)
            # check if the job is finished
            job_status = preprocess_functions.check_job_status(job)
            job_list[n]['Status'] = job_status
    # clear the job_list and schedule_table
    #job_list = []
    #schedule_table = pd.DataFrame()
    
# add function to sub_selector
sub_selector.observe(update_sub, names='value')

# add function to process_selector
process_selector.observe(process_selector_button_pressed, names='value')

# add function to run_selector
for run in run_selector.children:
    run.observe(update_runs, names='value')

# attach function to run_button
run_button.on_click(add_job_button_pressed)

# attach function to run_jobs_button
run_jobs_button.on_click(run_jobs)

# Create Scheduler tab
scheduler_tab = widgets.VBox([
    sub_selector,
    process_selector,
    run_selector,
    run_button,
    run_jobs_button,
    table_widget,
    ])

#tab_crop_app,crop_app_out = preprocess_functions.crop_app(project_dict, job_dict['sub_N'])
tab_crop_app,crop_app_out = preprocess_functions.crop_appB(project_dict)

## HERE###########
bet_app_tab, bet_app_out = preprocess_functions.bet_app(project_dict, job_dict['sub_N'])
 # Organize contents into tabs
tab_contents = ['Data Entry', 'Scheduler', 'Crop app', 'BET app']
tab_contents = ['Data Entry', 'Scheduler', 'Crop app']
# tab_list = [data_entry_tab,
#             scheduler_tab,
#              tab_crop_app,
#              bet_app_tab,
#              ]


tab_list = [data_entry_tab,
            scheduler_tab,
            tab_crop_app,
             ]
## HERE###########

# Organize contents into tabs
#tab_contents = ['Data Entry', 'Scheduler', 'Crop app']
#tab_list = [scheduler_tab,
#            data_entry_tab, 
#            tab_crop_app,
#            ]

tab.children = tab_list

for i, name in enumerate(tab_contents):
    tab.set_title(i, name)

# Function to handle tab change
def on_tab_change(change):
    if change['name'] == 'selected_index' and change['new'] == 2:
        print('hit 2')
        crop_app_pushed()
    elif change['name'] == 'selected_index' and change['new'] == 3:
        print('hit 3')
        bet_app_pushed()
    elif change['name'] == 'selected_index' and change['new'] == 0:
        print('hit 1')
        crop_app_out.clear_output()
        other_pushed()
        
# Observe changes in the tab widget
tab.observe(on_tab_change, names='selected_index')

# Display the tab widget
display(tab,crop_app_out,bet_app_out)

crop_app_out.clear_output()
bet_app_out.clear_output()


FileNotFoundError: No such file or no access: 'P:\userdata\raulh87\data\Complex\preprocessing\D-sub-001\D-sub-001_task-visual_mean_fct_uncut.nii.gz'

# Crop app

In [39]:
sub_N = 2
tab_crop_app,out = preprocess_functions.crop_app(project_dict, sub_N)
display(tab_crop_app,out)

Output()

# BET app

In [41]:
sub_N = 1
tab_crop_app,out = preprocess_functions.bet_app(project_dict, sub_N)
display(tab_crop_app,out)

Output()